In [20]:
from pathlib import Path
import csv
import os
from tqdm import tqdm

In [21]:
all_mp4s = os.popen('ls -d /data/dataset/babel_v1.0_release/renders/**/*.mp4').read().split()
all_mp4s = [Path(p) for p in all_mp4s]

In [27]:
def fast_scandir(dirname):
    subfolders= [Path(f.path) for f in os.scandir(dirname) if f.is_dir()]
    for dirname in list(subfolders):
        subfolders.extend(fast_scandir(dirname))
    return subfolders

def wrapper(p_files):
    return {p.stem: p.parent.name for p in p_files if p.suffix == '.mp4' and 'extra' not in p.parent.name}

start_index = 0
p_babel_filelist_dir = Path('/data/hyogun/repos/haawron_mmaction2/data/_filelists/babel')
p_babel_filelist_new_dir = p_babel_filelist_dir / 'processed'
p_babel_filelist_new_dir.mkdir(exist_ok=True)
p_babel_rawframes_dir = Path('/data/hyogun/repos/haawron_mmaction2/data/babel_v1.0_release/rawframes')
p_babel_videos_visited = wrapper(all_mp4s)
print(list(p_babel_videos_visited.keys())[:5])
for p_babel_filelist_old in p_babel_filelist_dir.glob('*.txt'):
    if 'stats' in p_babel_filelist_old.stem: continue
    p_babel_filelist_new = p_babel_filelist_new_dir / p_babel_filelist_old.name
    with p_babel_filelist_old.open('r') as f1, p_babel_filelist_new.open('w') as f2:
        reader = csv.reader(f1, delimiter=' ')
        writer = csv.writer(f2, delimiter=' ')
        for videoname, t_start, length, label in tqdm(list(reader)):
            fake_split, videoname = videoname.split('/')
            videoname = f'{int(videoname):06d}'
            real_split = p_babel_videos_visited[videoname]
            p_video = p_babel_rawframes_dir / real_split / videoname
            num_frames = len(list(p_video.glob('*.jpg')))
            t_start = max(int(t_start), start_index)
            length = min(int(length), num_frames-t_start)
            row = [f'{real_split}/{videoname}', t_start, length, label]
            writer.writerow(row)

['000001', '000005', '000006', '000009', '000014']


100%|██████████| 1672/1672 [00:03<00:00, 514.83it/s]
